<a href="https://colab.research.google.com/github/hakim3189/MachineLearning/blob/main/Sentiment_Analysis_Review_APK_Play_Store.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install google-play-scraper

# Mengimpor pustaka google_play_scraper untuk mengakses ulasan dan informasi aplikasi dari Google Play Store.
from google_play_scraper import app, reviews, Sort, reviews_all

import pandas as pd  # Pandas untuk manipulasi dan analisis data
pd.options.mode.chained_assignment = None  # Menonaktifkan peringatan chaining
import numpy as np  # NumPy untuk komputasi numerik
seed = 0
np.random.seed(seed)  # Mengatur seed untuk reproduktibilitas
import matplotlib.pyplot as plt  # Matplotlib untuk visualisasi data
import seaborn as sns  # Seaborn untuk visualisasi data statistik, mengatur gaya visualisasi
from sklearn.metrics import accuracy_score

import datetime as dt  # Manipulasi data waktu dan tanggal
import re  # Modul untuk bekerja dengan ekspresi reguler
import string  # Berisi konstanta string, seperti tanda baca
from nltk.tokenize import word_tokenize  # Tokenisasi teks
from nltk.corpus import stopwords  # Daftar kata-kata berhenti dalam teks

!pip install sastrawi
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory  # Stemming (penghilangan imbuhan kata) dalam bahasa Indonesia
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory  # Menghapus kata-kata berhenti dalam bahasa Indonesia

from wordcloud import WordCloud  # Membuat visualisasi berbentuk awan kata (word cloud) dari teks

import nltk  # Import pustaka NLTK (Natural Language Toolkit).
nltk.download('punkt')  # Mengunduh dataset yang diperlukan untuk tokenisasi teks.
nltk.download('stopwords')  # Mengunduh dataset yang berisi daftar kata-kata berhenti (stopwords) dalam berbagai bahasa.

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 5.2 MB/s eta 0:00:00


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [2]:
# Mengimpor pustaka google_play_scraper untuk mengakses ulasan dan informasi aplikasi dari Google Play Store.
from google_play_scraper import app, reviews_all, Sort

# Mengambil semua ulasan dari aplikasi dengan ID 'com.byu.id' di Google Play Store.
# Proses scraping mungkin memerlukan beberapa saat tergantung pada jumlah ulasan yang ada.
scrapreview = reviews_all(
    'com.byu.id',          # ID aplikasi
    lang='id',             # Bahasa ulasan (default: 'en')
    country='id',          # Negara (default: 'us')
    sort=Sort.MOST_RELEVANT, # Urutan ulasan (default: Sort.MOST_RELEVANT)
    count=1000             # Jumlah maksimum ulasan yang ingin diambil
)

In [3]:
import csv

with open('ulasan_aplikasi.csv', mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['Review'])  # Menulis header kolom
    for review in scrapreview:
        writer.writerow([review['content']])  # Menulis konten ulasan ke dalam file CSV

In [4]:
app_reviews_df = pd.DataFrame(scrapreview)
app_reviews_df.shape
app_reviews_df.head()
app_reviews_df.to_csv('ulasan_aplikasi.csv', index=False)

# Membuat DataFrame dari hasil scrapreview
app_reviews_df = pd.DataFrame(scrapreview)

# Menghitung jumlah baris dan kolom dalam DataFrame
jumlah_ulasan, jumlah_kolom = app_reviews_df.shape

In [5]:
app_reviews_df.head()

,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt,appVersion
0,9ed147fb-882e-4f83-9372-5b1cce1d78cf,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,"Jaringan sangat buruk, bahkan untuk membuka so...",1,30,1.60.0,2025-01-25 14:33:59,"Hai Kak, maaf nih terkait kendalanya, bisa cob...",2025-01-25 22:19:22,1.60.0
1,ebe4c7ae-c12c-4bba-bf37-7e90710b1142,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,UI dah lumayan tetapi UX-nya SAMPAH!! Setiap m...,3,22,1.60.0,2025-01-16 16:19:22,"Hi, Kak Naufal. Maaf banget udah bikin gak nya...",2025-01-16 22:13:55,1.60.0
2,c60230a6-b4b5-4f32-8dcb-0438e9ba00be,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,Operator ini tinggal tunggu waktu buat bubar. ...,1,49,1.60.0,2025-01-23 00:53:42,"Hi, Kak. Maaf banget jadi bikin gak nyaman. Ni...",2025-01-23 02:16:47,1.60.0
3,4160042b-9665-400e-8d97-486f9041e86a,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,"Kouta boros, jaringan tidak stabil, pulsa kepo...",1,100,1.60.0,2025-01-17 03:21:29,"Hi, Kak. Maaf banget jadi bikin gak nyaman. Ni...",2025-01-17 05:19:12,1.60.0
4,1afa8a8d-6891-4b41-b84e-03a59c911d57,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,Aplikasi serring error ada pemberitahuan sedan...,5,84,1.60.0,2025-01-20 14:35:47,"Hi, Kak. Maaf banget jadi bikin gak nyaman. Ni...",2025-01-21 01:45:48,1.60.0


In [6]:
app_reviews_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 157753 entries, 0 to 157752
Data columns (total 11 columns):
 #   Column                Non-Null Count   Dtype         
---  ------                --------------   -----         
 0   reviewId              157753 non-null  object        
 1   userName              157753 non-null  object        
 2   userImage             157753 non-null  object        
 3   content               157752 non-null  object        
 4   score                 157753 non-null  int64         
 5   thumbsUpCount         157753 non-null  int64         
 6   reviewCreatedVersion  133042 non-null  object        
 7   at                    157753 non-null  datetime64[ns]
 8   replyContent          139281 non-null  object        
 9   repliedAt             139281 non-null  datetime64[ns]
 10  appVersion            133042 non-null  object        
dtypes: datetime64[ns](2), int64(2), object(7)
memory usage: 13.2+ MB


In [7]:
clean_df = app_reviews_df.dropna()

In [8]:
# Menghapus baris duplikat dari DataFrame clean_df
clean_df = clean_df.drop_duplicates()

# Menghitung jumlah baris dan kolom dalam DataFrame clean_df setelah menghapus duplikat
jumlah_ulasan_setelah_hapus_duplikat, jumlah_kolom_setelah_hapus_duplikat = clean_df.shape

In [9]:
import re
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
nltk.download('punkt_tab')


def cleaningText(text):
    text = re.sub(r'@[A-Za-z0-9]+', '', text) # menghapus mention
    text = re.sub(r'#[A-Za-z0-9]+', '', text) # menghapus hashtag
    text = re.sub(r'RT[\s]', '', text) # menghapus RT
    text = re.sub(r"http\S+", '', text) # menghapus link
    text = re.sub(r'[0-9]+', '', text) # menghapus angka
    text = re.sub(r'[^\w\s]', '', text) # menghapus karakter selain huruf dan angka

    text = text.replace('\n', ' ') # mengganti baris baru dengan spasi
    text = text.translate(str.maketrans('', '', string.punctuation)) # menghapus semua tanda baca
    text = text.strip(' ') # menghapus karakter spasi dari kiri dan kanan teks
    return text

def casefoldingText(text): # Mengubah semua karakter dalam teks menjadi huruf kecil
    text = text.lower()
    return text

def tokenizingText(text): # Memecah atau membagi string, teks menjadi daftar token
    text = word_tokenize(text)
    return text

def filteringText(text): # Menghapus stopwords dalam teks
    listStopwords = set(stopwords.words('indonesian'))
    listStopwords1 = set(stopwords.words('english'))
    listStopwords.update(listStopwords1)
    listStopwords.update(['iya','yaa','gak','nya','na','sih','ku',"di","ga","ya","gaa","loh","kah","woi","woii","woy"])
    filtered = []
    for txt in text:
        if txt not in listStopwords:
            filtered.append(txt)
    text = filtered
    return text

def stemmingText(text): # Mengurangi kata ke bentuk dasarnya yang menghilangkan imbuhan awalan dan akhiran atau ke akar kata
    # Membuat objek stemmer
    factory = StemmerFactory()
    stemmer = factory.create_stemmer()

    # Memecah teks menjadi daftar kata
    words = text.split()

    # Menerapkan stemming pada setiap kata dalam daftar
    stemmed_words = [stemmer.stem(word) for word in words]

    # Menggabungkan kata-kata yang telah distem
    stemmed_text = ' '.join(stemmed_words)

    return stemmed_text

def toSentence(list_words): # Mengubah daftar kata menjadi kalimat
    sentence = ' '.join(word for word in list_words)
    return sentence

In [11]:
slangwords = {"@": "di", "abis": "habis", "wtb": "beli", "masi": "masih", "wts": "jual", "wtt": "tukar", "bgt": "banget", "maks": "maksimal"}
def fix_slangwords(text):
    words = text.split()
    fixed_words = []

    for word in words:
        if word.lower() in slangwords:
            fixed_words.append(slangwords[word.lower()])
        else:
            fixed_words.append(word)

    fixed_text = ' '.join(fixed_words)
    return fixed_text

In [13]:
import nltk
nltk.download('punkt_tab')
# Membersihkan teks dan menyimpannya di kolom 'text_clean'
clean_df['text_clean'] = clean_df['content'].apply(cleaningText)

# Mengubah huruf dalam teks menjadi huruf kecil dan menyimpannya di 'text_casefoldingText'
clean_df['text_casefoldingText'] = clean_df['text_clean'].apply(casefoldingText)

# Mengganti kata-kata slang dengan kata-kata standar dan menyimpannya di 'text_slangwords'
clean_df['text_slangwords'] = clean_df['text_casefoldingText'].apply(fix_slangwords)

# Memecah teks menjadi token (kata-kata) dan menyimpannya di 'text_tokenizingText'
clean_df['text_tokenizingText'] = clean_df['text_slangwords'].apply(tokenizingText)

# Menghapus kata-kata stop (kata-kata umum) dan menyimpannya di 'text_stopword'
clean_df['text_stopword'] = clean_df['text_tokenizingText'].apply(filteringText)

# Menggabungkan token-token menjadi kalimat dan menyimpannya di 'text_akhir'
clean_df['text_akhir'] = clean_df['text_stopword'].apply(toSentence)

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [14]:
clean_df

,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt,appVersion,text_clean,text_casefoldingText,text_slangwords,text_tokenizingText,text_stopword,text_akhir
0,9ed147fb-882e-4f83-9372-5b1cce1d78cf,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,"Jaringan sangat buruk, bahkan untuk membuka so...",1,30,1.60.0,2025-01-25 14:33:59,"Hai Kak, maaf nih terkait kendalanya, bisa cob...",2025-01-25 22:19:22,1.60.0,Jaringan sangat buruk bahkan untuk membuka sos...,jaringan sangat buruk bahkan untuk membuka sos...,jaringan sangat buruk bahkan untuk membuka sos...,"[jaringan, sangat, buruk, bahkan, untuk, membu...","[jaringan, buruk, membuka, sosmed, sejenis, ig...",jaringan buruk membuka sosmed sejenis ig tikto...
1,ebe4c7ae-c12c-4bba-bf37-7e90710b1142,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,UI dah lumayan tetapi UX-nya SAMPAH!! Setiap m...,3,22,1.60.0,2025-01-16 16:19:22,"Hi, Kak Naufal. Maaf banget udah bikin gak nya...",2025-01-16 22:13:55,1.60.0,UI dah lumayan tetapi UXnya SAMPAH Setiap meng...,ui dah lumayan tetapi uxnya sampah setiap meng...,ui dah lumayan tetapi uxnya sampah setiap meng...,"[ui, dah, lumayan, tetapi, uxnya, sampah, seti...","[ui, dah, lumayan, uxnya, sampah, misi, harian...",ui dah lumayan uxnya sampah misi harian ustamp...
2,c60230a6-b4b5-4f32-8dcb-0438e9ba00be,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,Operator ini tinggal tunggu waktu buat bubar. ...,1,49,1.60.0,2025-01-23 00:53:42,"Hi, Kak. Maaf banget jadi bikin gak nyaman. Ni...",2025-01-23 02:16:47,1.60.0,Operator ini tinggal tunggu waktu buat bubar K...,operator ini tinggal tunggu waktu buat bubar k...,operator ini tinggal tunggu waktu buat bubar k...,"[operator, ini, tinggal, tunggu, waktu, buat, ...","[operator, tinggal, tunggu, bubar, kualitas, j...",operator tinggal tunggu bubar kualitas jaringa...
3,4160042b-9665-400e-8d97-486f9041e86a,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,"Kouta boros, jaringan tidak stabil, pulsa kepo...",1,100,1.60.0,2025-01-17 03:21:29,"Hi, Kak. Maaf banget jadi bikin gak nyaman. Ni...",2025-01-17 05:19:12,1.60.0,Kouta boros jaringan tidak stabil pulsa kepoto...,kouta boros jaringan tidak stabil pulsa kepoto...,kouta boros jaringan tidak stabil pulsa kepoto...,"[kouta, boros, jaringan, tidak, stabil, pulsa,...","[kouta, boros, jaringan, stabil, pulsa, kepoto...",kouta boros jaringan stabil pulsa kepotong cum...
4,1afa8a8d-6891-4b41-b84e-03a59c911d57,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,Aplikasi serring error ada pemberitahuan sedan...,5,84,1.60.0,2025-01-20 14:35:47,"Hi, Kak. Maaf banget jadi bikin gak nyaman. Ni...",2025-01-21 01:45:48,1.60.0,Aplikasi serring error ada pemberitahuan sedan...,aplikasi serring error ada pemberitahuan sedan...,aplikasi serring error ada pemberitahuan sedan...,"[aplikasi, serring, error, ada, pemberitahuan,...","[aplikasi, serring, error, pemberitahuan, rame...",aplikasi serring error pemberitahuan rame yg a...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
151085,cd5f5eaf-f7fa-4619-b4ec-4f8f627d0cbe,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,Oke,5,0,1.0.318 - by.U,2019-10-25 12:44:20,Hi kak makasi yah buat support nya :),2019-10-28 02:42:49,1.0.318 - by.U,Oke,oke,oke,[oke],[oke],oke
151089,5d3c977c-f441-4278-be52-69d670684a58,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,Ga bsa dafatar via fb,1,1,1.0.313,2019-10-19 09:02:35,"hai hai, untuk troubleshoot solution bisa hubu...",2019-10-19 11:22:14,1.0.313,Ga bsa dafatar via fb,ga bsa dafatar via fb,ga bsa dafatar via fb,"[ga, bsa, dafatar, via, fb]","[bsa, dafatar, via, fb]",bsa dafatar via fb
151090,a81ed629-6cc7-4a68-8d9f-fe527924dab2,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,Gak bisa sign up atau log in,1,1,1.0.313,2019-10-18 19:43:03,"Hi kak,\nNindy mohon maap yaaa, kita emang sem...",201

Pelabelan

In [15]:
import csv
import requests
from io import StringIO

# Membaca data kamus kata-kata positif dari GitHub
lexicon_positive = dict()

response = requests.get('https://raw.githubusercontent.com/angelmetanosaa/dataset/main/lexicon_positive.csv')
# Mengirim permintaan HTTP untuk mendapatkan file CSV dari GitHub

if response.status_code == 200:
    # Jika permintaan berhasil
    reader = csv.reader(StringIO(response.text), delimiter=',')
    # Membaca teks respons sebagai file CSV menggunakan pembaca CSV dengan pemisah koma

    for row in reader:
        # Mengulangi setiap baris dalam file CSV
        lexicon_positive[row[0]] = int(row[1])
        # Menambahkan kata-kata positif dan skornya ke dalam kamus lexicon_positive
else:
    print("Failed to fetch positive lexicon data")

# Membaca data kamus kata-kata negatif dari GitHub
lexicon_negative = dict()

response = requests.get('https://raw.githubusercontent.com/angelmetanosaa/dataset/main/lexicon_negative.csv')
# Mengirim permintaan HTTP untuk mendapatkan file CSV dari GitHub

if response.status_code == 200:
    # Jika permintaan berhasil
    reader = csv.reader(StringIO(response.text), delimiter=',')
    # Membaca teks respons sebagai file CSV menggunakan pembaca CSV dengan pemisah koma

    for row in reader:
        # Mengulangi setiap baris dalam file CSV
        lexicon_negative[row[0]] = int(row[1])
        # Menambahkan kata-kata negatif dan skornya dalam kamus lexicon_negative
else:
    print("Failed to fetch negative lexicon data")

In [16]:
def sentiment_analysis_lexicon_indonesia(text):
    #for word in text:

    score = 0
    # Inisialisasi skor sentimen ke 0

    for word in text:
        # Mengulangi setiap kata dalam teks

        if (word in lexicon_positive):
            score = score + lexicon_positive[word]
            # Jika kata ada dalam kamus positif, tambahkan skornya ke skor sentimen

    for word in text:
        # Mengulangi setiap kata dalam teks (sekali lagi)

        if (word in lexicon_negative):
            score = score + lexicon_negative[word]
            # Jika kata ada dalam kamus negatif, kurangkan skornya dari skor sentimen

    polarity=''
    # Inisialisasi variabel polaritas

    if (score >= 0):
        polarity = 'positive'
        # Jika skor sentimen lebih besar atau sama dengan 0, maka polaritas adalah positif
    elif (score < 0):
        polarity = 'negative'
        # Jika skor sentimen kurang dari 0, maka polaritas adalah negatif

    # else:
    #     polarity = 'neutral'
    # Ini adalah bagian yang bisa digunakan untuk menentukan polaritas netral jika diperlukan

    return score, polarity
    # Mengembalikan skor sentimen dan polaritas teks

In [17]:
results = clean_df['text_stopword'].apply(sentiment_analysis_lexicon_indonesia)
results = list(zip(*results))
clean_df['polarity_score'] = results[0]
clean_df['polarity'] = results[1]
print(clean_df['polarity'].value_counts())

polarity
positive    68878
negative    48075
Name: count, dtype: int64


Data Splitting dan Ekstraksi Fitur dengan TF-IDF

In [19]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split


# Pisahkan data menjadi fitur (tweet) dan label (sentimen)
X = clean_df['text_akhir']
y = clean_df['polarity']

# Ekstraksi fitur dengan TF-IDF
tfidf = TfidfVectorizer(max_features=200, min_df=17, max_df=0.8 )
X_tfidf = tfidf.fit_transform(X)

# Konversi hasil ekstraksi fitur menjadi dataframe
features_df = pd.DataFrame(X_tfidf.toarray(), columns=tfidf.get_feature_names_out())

# Menampilkan hasil ekstraksi fitur
features_df

# Bagi data menjadi data latih dan data uji
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y, test_size=0.2, random_state=42)

Naive Baiyes

In [21]:
from sklearn.naive_bayes import BernoulliNB # Import the BernoulliNB class

# Membuat objek model Naive Bayes (Bernoulli Naive Bayes)
naive_bayes = BernoulliNB()

# Melatih model Naive Bayes pada data pelatihan
naive_bayes.fit(X_train.toarray(), y_train)

# Prediksi sentimen pada data pelatihan dan data uji
y_pred_train_nb = naive_bayes.predict(X_train.toarray())
y_pred_test_nb = naive_bayes.predict(X_test.toarray())

# Evaluasi akurasi model Naive Bayes
accuracy_train_nb = accuracy_score(y_pred_train_nb, y_train)
accuracy_test_nb = accuracy_score(y_pred_test_nb, y_test)

# Menampilkan akurasi
print('Naive Bayes - accuracy_train:', accuracy_train_nb)
print('Naive Bayes - accuracy_test:', accuracy_test_nb)

Naive Bayes - accuracy_train: 0.7852226331202838
Naive Bayes - accuracy_test: 0.7839767431918259


Random Forest

In [22]:
from sklearn.ensemble import RandomForestClassifier

# Membuat objek model Random Forest
random_forest = RandomForestClassifier()

# Melatih model Random Forest pada data pelatihan
random_forest.fit(X_train.toarray(), y_train)

# Prediksi sentimen pada data pelatihan dan data uji
y_pred_train_rf = random_forest.predict(X_train.toarray())
y_pred_test_rf = random_forest.predict(X_test.toarray())

# Evaluasi akurasi model Random Forest
accuracy_train_rf = accuracy_score(y_pred_train_rf, y_train)
accuracy_test_rf = accuracy_score(y_pred_test_rf, y_test)

# Menampilkan akurasi
print('Random Forest - accuracy_train:', accuracy_train_rf)
print('Random Forest - accuracy_test:', accuracy_test_rf)

Random Forest - accuracy_train: 0.9730873645283341
Random Forest - accuracy_test: 0.8926510196229319


Logistic Regression

In [23]:
from sklearn.linear_model import LogisticRegression

# Membuat objek model Logistic Regression
logistic_regression = LogisticRegression()

# Melatih model Logistic Regression pada data pelatihan
logistic_regression.fit(X_train.toarray(), y_train)

# Prediksi sentimen pada data pelatihan dan data uji
y_pred_train_lr = logistic_regression.predict(X_train.toarray())
y_pred_test_lr = logistic_regression.predict(X_test.toarray())

# Evaluasi akurasi model Logistic Regression pada data pelatihan
accuracy_train_lr = accuracy_score(y_pred_train_lr, y_train)

# Evaluasi akurasi model Logistic Regression pada data uji
accuracy_test_lr = accuracy_score(y_pred_test_lr, y_test)

# Menampilkan akurasi
print('Logistic Regression - accuracy_train:', accuracy_train_lr)
print('Logistic Regression - accuracy_test:', accuracy_test_lr)

Logistic Regression - accuracy_train: 0.9114918449797995
Logistic Regression - accuracy_test: 0.905989483134539


Decision Tree

In [24]:
from sklearn.tree import DecisionTreeClassifier

# Membuat objek model Decision Tree
decision_tree = DecisionTreeClassifier()

# Melatih model Decision Tree pada data pelatihan
decision_tree.fit(X_train.toarray(), y_train)

# Prediksi sentimen pada data pelatihan dan data uji
y_pred_train_dt = decision_tree.predict(X_train.toarray())
y_pred_test_dt = decision_tree.predict(X_test.toarray())

# Evaluasi akurasi model Decision Tree
accuracy_train_dt = accuracy_score(y_pred_train_dt, y_train)
accuracy_test_dt = accuracy_score(y_pred_test_dt, y_test)

# Menampilkan akurasi
print('Decision Tree - accuracy_train:', accuracy_train_dt)
print('Decision Tree - accuracy_test:', accuracy_test_dt)

Decision Tree - accuracy_train: 0.9730873645283341
Decision Tree - accuracy_test: 0.8607584113547946
